In [288]:
import pandas as pd
import urllib.request
import time
import json

# KMDb OPEN API

In [289]:
movie = pd.read_csv('../data/db2_NaN_csv/film_festival_2000.csv', encoding ='utf-8')
#데이터 엑셀 파일 출처 : https://www.kobis.or.kr/kobis/business/mast/fest/searchUserFestInfoList.do 한국진흥원
client_id =''
url = 'http://api.koreafilm.or.kr/openapi-data2/wisenut/search_api/search_json2.jsp?collection=kmdb_new2'
option = "&listCount=500&detail=Y&ServiceKey=IG9S02XET4ZZ509DATVC"
movie

,연도,영화제,장르,출품작,감독,출품분야,수상내역,수상인
0,2000,타이베이 영화제,일반(종합),물안경,이수연,Focus on Korean CInema,NaN,NaN
1,2000,타이베이 영화제,일반(종합),자반고등어,김용화,Focus on Korean CInema,NaN,NaN
2,2000,타이베이 영화제,일반(종합),오! 수정,홍상수,Focus on Korean CInema,NaN,NaN
3,2000,타이베이 영화제,일반(종합),섬,김기덕,Focus on Korean CInema,NaN,NaN
4,2000,타이베이 영화제,일반(종합),가위,이기철,Focus on Korean CInema,NaN,NaN
...,...,...,...,...,...,...,...,...
349,2000,디렉터스 컷 어워즈,일반(종합),공동경비구역 JSA,박찬욱,NaN,올해의 감독상,박찬욱
350,2000,디렉터스 컷 어워즈,일반(종합),공동경비구역 JSA,박찬욱,NaN,올해의 제작자상,심재명
351,2000,디렉터스 컷 어워즈,일반(종합),공동경비구역 JSA,박찬욱,NaN,올해의 남자배우상,송강호
352,2000,디렉터스 컷 어워즈,일반(종합),공동경비구역 JSA,박찬욱,NaN,올해의 새로운 남자배우상,신하균


In [290]:
movie = movie.drop_duplicates(['출품작', '감독'], ignore_index=True)
#한 작품이 여러 영화제에 나가서 중복된 데이터가 많으므로 중복된 값들 제거해준다.

### 데이터 전처리 과정
- 감독이 NaN 인 row 삭제, 수상인 column 삭제, 독립영화에 포커스를 하는 것이므로 영화제에 독립, 단편이 포함된 영화제만 걸러낸다.
- columns 이름들을 다 영어로 바꿔준 뒤에 director에 ,가 포함된 2명 이상의 감독 제외 (코드 상의 어려움, 추후 수정 요망)
- index를 새로 맞춰준다. 

In [291]:
movie_Na = movie.dropna(subset=['감독'])
movie_Na = movie_Na.drop('수상인', axis =1 )
movie_Na = movie_Na.query('영화제.str.contains("독립|단편|음식")', engine='python')
movie_Na = movie_Na.rename(columns={'연도':'year','영화제':'film_festival','장르':'genre','출품작':'title','감독':'director','출품분야':'field','수상내역':'award'})
movie_Na = movie_Na[~movie_Na['director'].str.contains(",", na=False, case=False)]
movie_Na = movie_Na.reset_index()
movie_Na = movie_Na.drop(columns=['index'])
movie_Na

,year,film_festival,genre,title,director,field,award
0,2000,서울독립영화제,독립,진공의 시간,정사람,본선경쟁,우수작품상
1,2000,서울독립영화제,독립,구타 유발자 잠들다,유정현,본선경쟁,최우수작품상
2,2000,서울독립영화제,독립,언년이,유진희,본선경쟁,우수작품상
3,2000,서울독립영화제,독립,팬지와 담쟁이,계운경,본선경쟁,우수작품상
4,2000,서울독립영화제,독립,인간의 시간,태준식,본선경쟁,우수작품상
5,2000,서울독립영화제,독립,존재,이명하,본선경쟁,최우수작품상
6,2000,서울독립영화제,독립,대학로에서 매춘하다가 토막살해당한 여고생 아직 대학로에 있다,남기웅,본선경쟁,우수작품상
7,2000,대구단편영화제,단편,이발소,이기호,NaN,대상
8,2000,대구단편영화제,단편,해發芽記,주재형,NaN,촬영상
9,2000,대구단편영화제,단편,출근,서원태,NaN,연기상


In [292]:
movie = movie_Na.drop(columns=['genre'])
movie

,year,film_festival,title,director,field,award
0,2000,서울독립영화제,진공의 시간,정사람,본선경쟁,우수작품상
1,2000,서울독립영화제,구타 유발자 잠들다,유정현,본선경쟁,최우수작품상
2,2000,서울독립영화제,언년이,유진희,본선경쟁,우수작품상
3,2000,서울독립영화제,팬지와 담쟁이,계운경,본선경쟁,우수작품상
4,2000,서울독립영화제,인간의 시간,태준식,본선경쟁,우수작품상
5,2000,서울독립영화제,존재,이명하,본선경쟁,최우수작품상
6,2000,서울독립영화제,대학로에서 매춘하다가 토막살해당한 여고생 아직 대학로에 있다,남기웅,본선경쟁,우수작품상
7,2000,대구단편영화제,이발소,이기호,NaN,대상
8,2000,대구단편영화제,해發芽記,주재형,NaN,촬영상
9,2000,대구단편영화제,출근,서원태,NaN,연기상


In [293]:
plot = []
rating = []
runtime = []
prodYear = []
genre = []
poster = []

for i in range(0,len(movie_Na.title)):
    query = "&title="+urllib.parse.quote(movie_Na.title[i])+"&director="+urllib.parse.quote(movie_Na.director[i])
    url_query = url + query + option
    request = urllib.request.Request(url_query)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode == 200):
        response_body = json.load(response)
    else:
        print("Error code:"+rescode)
    if 'Result' in response_body['Data'][0]:
        for j in range(0,len(response_body['Data'][0]['Result'])):
            if movie_Na.title[i][0:3] in response_body['Data'][0]['Result'][j]['titleEtc']:
                plot.append(response_body['Data'][0]['Result'][j]['plots']['plot'][0]['plotText'])
                runtime.append(response_body['Data'][0]['Result'][j]['runtime'])
                prodYear.append(response_body['Data'][0]['Result'][j]['prodYear'])
                rating.append(response_body['Data'][0]['Result'][j]['rating'])
                genre.append(response_body['Data'][0]['Result'][j]['genre'])
                poster.append(response_body['Data'][0]['Result'][j]['posters'])
                break
            else:
                plot.append('')
                runtime.append('')
                prodYear.append('')
                poster.append('')
                rating.append('')
                genre.append('기타')                
                break
    else:
        plot.append('')
        runtime.append('')
        prodYear.append('')
        rating.append('')
        poster.append('')
        genre.append('기타')
        
for idx, x in enumerate(genre):
    if x == '':
        genre[idx] = '기타'

In [294]:
temp_= []
stills = []
for i in range(0,len(movie_Na.title)):
    query = "&title="+urllib.parse.quote(movie_Na.title[i])+"&director="+urllib.parse.quote(movie_Na.director[i])
    url_query = url + query + option
    request = urllib.request.Request(url_query)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode == 200):
        response_body = json.load(response)
    else:
        print("Error code:"+rescode)
    if 'Result' in response_body['Data'][0]:
        for j in range(0, len(response_body['Data'][0]['Result'])):
            if movie_Na.title[i][0:3] in response_body['Data'][0]['Result'][j]['titleEtc']:
                temp_.append(response_body['Data'][0]['Result'][j]['stlls'])
                break
            else :
                temp_.append('')
                break
    else:
        temp_.append('')
for x in temp_:
    stills.append(x.replace('|',','))

In [295]:
actors_temp = []

for i in range(0,len(movie_Na.title)):
    query = "&title="+urllib.parse.quote(movie_Na.title[i])+"&director="+urllib.parse.quote(movie_Na.director[i])
    url_query = url + query + option
    request = urllib.request.Request(url_query)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode == 200):
        response_body = json.load(response)
    else:
        print("Error code:"+rescode)
    if 'Result' in response_body['Data'][0]:
        for j in range(0, len(response_body['Data'][0]['Result'])):
            if movie_Na.title[i][0:3] in response_body['Data'][0]['Result'][j]['titleEtc']:
                temp = []
                for k in range(0,len(response_body['Data'][0]['Result'][j]['actors']['actor'])):
                    temp.append(response_body['Data'][0]['Result'][j]['actors']['actor'][k]['actorNm'])
                temp = temp[0:3]
                actors_temp.append(temp)
                break
            else :
                actors_temp.append('')
                break
    else:
        actors_temp.append('')
actors = []
for x in actors_temp:
    str = ','.join(x)
    actors.append(str)

# NAVER API
- img link만 추출하기 위함, KMDb에는 포스터에 대한 데이터가 매우 적다.

In [296]:
client_id =''
client_secret = ''
url = "https://openapi.naver.com/v1/search/movie.json"
option = "&display=100"
movie.head()

,year,film_festival,title,director,field,award
0,2000,서울독립영화제,진공의 시간,정사람,본선경쟁,우수작품상
1,2000,서울독립영화제,구타 유발자 잠들다,유정현,본선경쟁,최우수작품상
2,2000,서울독립영화제,언년이,유진희,본선경쟁,우수작품상
3,2000,서울독립영화제,팬지와 담쟁이,계운경,본선경쟁,우수작품상
4,2000,서울독립영화제,인간의 시간,태준식,본선경쟁,우수작품상


In [297]:
img_link_temp = []
for i in range(0, len(movie.title)):
    query = "?query="+urllib.parse.quote(movie.title[i])
    url_query = url + query + option
    request = urllib.request.Request(url_query)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    rescode = response.getcode()
    if(rescode == 200):
        response_body = json.load(response)
    else:
        print("Error code:"+rescode)
    if len(response_body['items']) != 0:
        for j in range(0, len(response_body['items'])):
            if movie.title[i] in response_body['items'][j]['title'].strip('<b>') and movie.director[i] in response_body['items'][j]['director'] :
                img_link_temp.append(response_body['items'][j]['image'])
                img_link_temp.append(movie.director[i])
#                 print(response_body['items'][j]['title'])
                break
        if len(img_link_temp)-2 < (2 * i):
            img_link_temp.append('')
            img_link_temp.append(movie.director[i])

    else:
        img_link_temp.append('')
        img_link_temp.append(movie.director[i])
        
    time.sleep(0.1)
    
img_link = []
for idx, x in enumerate(img_link_temp) :
    if idx % 2 == 0 :
        img_link.append(x)
print(len(img_link))


26


### array로 만들어 놓은 컬럼을 데이터프레임에 추가해준다.

In [298]:
movie['genre'] = genre
movie['plot'] = plot
movie['rating'] = rating
movie['runtime'] = runtime
movie['prodYear'] = prodYear
movie['actors'] = actors
movie['img_link'] = img_link
movie['stills'] = stills

In [299]:
movie.to_csv('film_festival_2000.csv',encoding="utf8", index=False)